In [1]:
from selenium import webdriver
from selenium.webdriver import Safari, Chrome, Firefox
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import time
import pickle

In [2]:
def scroll_down(driver, func):
    """A method for scrolling the page."""

    reviews_list = test_count(driver.page_source)
    
    for i in range(3):
#     while True:
        end_element = driver.find_element(By.CLASS_NAME, "js--reviews--loadMore")
        
        target_y = end_element.location["y"]
        for distance in range(-10,10):
            driver.execute_script("window.scrollTo(0, "+str(target_y + distance)+")")
            time.sleep(1)

        # Wait to load the page.
        time.sleep(10)
        
        new_list = test_count(driver.page_source)

        if len(new_list) == len(reviews_list):

            break

        reviews_list = new_list
    
    return reviews_list

In [3]:
def test_count(page_source):
    soup = BeautifulSoup(page_source, "html.parser")
    reviews_list = soup.find_all(class_="review-stack--poster")
    print(len(reviews_list))
    return reviews_list

In [4]:
def count_stars(star_div):
    return len(star_div.find_all(class_="icon-star-full")) + 0.5 * len(star_div.find_all(class_="icon-star-half"))

def extract_ebert_info(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    
    # Title
    title_divs = [t.text.strip() for t in soup.find_all(class_="page-content--title")]
    assert(len(title_divs) == 1)
    title = title_divs[0]
    
    # Author
    author_divs = [t.text.strip() for t in soup.find_all(class_="byline")]
    assert(len(author_divs) == 1)
    author = author_divs[0]
    
    # Stars
    star_divs = soup.find_all(class_="page-content--star-rating")
    assert(len(star_divs) == 1)
    star_div = star_divs[0]

    star_count = count_stars(star_div)
    
    # Content
    content_paragraphs = []
    content_divs_tmp = soup.find_all(class_="page-content--block_editor-content")
    for c in content_divs_tmp:
        content_paragraphs.extend([t.text.strip() for t in c.find_all(["p", "li"])])
        
    return title, author, star_count, content_paragraphs

In [5]:
driver = Firefox()
driver.get("https://www.rogerebert.com/reviews")

reviews_links_list = scroll_down(driver, test_count)

driver.quit()

28
52
76
100


In [14]:
with open('raw_reviews.pkl', 'wb') as f:
    pickle.dump(reviews_links_list, f, pickle.HIGHEST_PROTOCOL)

In [7]:
reviews_links_list = list(set(reviews_links_list))

In [8]:
reviews_links_list = [r['href'] for r in reviews_links_list]

In [ ]:
dataset_rough = [extract_ebert_info(url) for url in reviews_links_list]

In [ ]:
with open('dataset_rough.pkl', 'wb') as f:
    pickle.dump(dataset_rough, f, pickle.HIGHEST_PROTOCOL)